## 연습문제 1
- 1 사전기반 분류분석 연습문제 2번의 수행 결과로 600 행의 태깅된 [사람 문장 1] 데이터를 모두 합쳐서 하나로 만듦
    - 조별작업_감정사전 > 감정대화라벨링말뭉치_3600.xlsx
- 2 합친 데이터를 7.5:2.5의 비율로 train – test 분리하여 train 데이터로 모
델을 만들고, test 데이터로 평가
    - 긍정-부정의 이진분류만 진행
    - 3개 이상의 알고리즘을 사용하며, K-fold 교차검증을 수행
    - 가장 좋은 성능을 보인 알고리즘으로 긍정-중립-부정의 다중분류도 진행
- 3 test 데이터에 대하여 사전기반 분류분석에서 만든 감성사전을 이용하여 감성분석을 수행하고, 머신러닝 모델의 성능과 비교

In [1]:
import pandas as pd
import numpy as np
from matplotlib import rc

rc('font', family='AppleGothic')

path = '/Users/jsha/gjai/nlp/pytest/'
file = '감정대화라벨링말뭉치_3600.csv'

df = pd.read_csv(path+file)

# print(len(df))
df = df[(df['결과']==-1.) | (df['결과']==1.)]
df['결과'].value_counts()

-1.0    2720
 1.0     469
Name: 결과, dtype: int64

In [2]:
from tqdm import tqdm # 이거 엄청 느리네. 쓰지말자.
import rhinoMorph

rn = rhinoMorph.startRhino()


stopwords_ko = ["하다", "있다", "되다", "그", "않다", "없다", "나", "말", 
                "사람", "이", "보다", " 한", "때", "년", "같다", "대하다", 
                "일", "이", "생각", "위하다", "때문", "그것", "그러나", "가다",
                " 받다", "그렇다", "알다", "사회", "더", "그녀", "문제", 
                "오다", "그리고", "크다", "속"]

def morph_list(string):
    morphed_list = rhinoMorph.onlyMorph_list(rn, string,
                    pos= ['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 
                          'MM', 'MAG', 'MAJ'], eomi=True)
    morphed_list = [text for text in morphed_list if text 
                         not in stopwords_ko]
    return ' '.join(morphed_list)

df['morph_str'] = df['사람문장1'].apply(morph_list)



filepath:  /Users/jsha/opt/anaconda3/envs/nlp3710/lib/python3.7/site-packages
classpath:  /Users/jsha/opt/anaconda3/envs/nlp3710/lib/python3.7/site-packages/rhinoMorph/lib/rhino.jar
RHINO started!


In [3]:
data_text = df['morph_str']
data_senti = df['결과']

# stratify를 사용한 것에 주목
from sklearn.model_selection import train_test_split
train_data_text, test_data_text, train_data_senti, test_data_senti = \
train_test_split(data_text, data_senti, stratify=data_senti, \
                 train_size=0.75, random_state=1111)


# Counter 클래스를 이용해 data_senti의 value 값과 수를 고려하여 잘 분배되었는지 확인
from collections import Counter
train_data_senti_freq = Counter(train_data_senti)
print('train_data_senti_freq:', train_data_senti_freq)


train_data_senti_freq: Counter({-1.0: 2039, 1.0: 352})


In [4]:
2039/681

2.9941262848751835

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(min_df=5).fit(train_data_text)
X_train = vect.transform(train_data_text)
print("X_train:\n", repr(X_train))
X_test = vect.transform(test_data_text)




X_train:
 <2391x570 sparse matrix of type '<class 'numpy.int64'>'
	with 10958 stored elements in Compressed Sparse Row format>


In [6]:
feature_names = vect.get_feature_names()
print('특성 갯수: ', len(feature_names))
print('20개 특성: ', feature_names[:20])

특성 갯수:  570
20개 특성:  ['가게', '가끔', '가난', '가입', '가장', '가족', '가지다', '가진', '가해자', '간호사', '감사', '감정', '갑자기', '갖다', '같이', '갚다', '거의', '거짓말', '걱정', '건강']


In [9]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression

y_train = pd.Series(train_data_senti)
y_test = pd.Series(test_data_senti)

lr = LogisticRegression(solver='liblinear')
splitter = StratifiedKFold(n_splits=4, shuffle=True, random_state=1111)
scores = cross_validate(lr, X_train, y_train, cv=splitter)
print('score:', np.mean(scores['test_score']))

lr.fit(X_train, y_train)
print('test_score: ', lr.score(X_test, y_test))

score: 0.8900039495134535
test_score:  0.8972431077694235


In [10]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier()
scores = cross_validate(gb, X_train, y_train, cv=splitter)
print(np.mean(scores['test_score']))

gb.fit(X_train, y_train)
print('test_score: ', gb.score(X_test, y_test))

0.8979498943995338
test_score:  0.899749373433584


In [11]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
scores = cross_validate(rf, X_train, y_train, cv=splitter)
print(np.mean(scores['test_score']))

rf.fit(X_train, y_train)
print('test_score: ', rf.score(X_test, y_test))

0.8762065623546943
test_score:  0.8696741854636592


In [13]:
from sklearn.svm import SVC

svc = SVC(kernel='linear')
scores = cross_validate(svc, X_train, y_train, cv=splitter)
# print(np.mean(scores['test_score']))

svc.fit(X_train, y_train)
svc_result = svc.predict(X_test)
print('test_score: ', svc.score(X_test, y_test))

test_score:  0.899749373433584


In [ ]:
# from lightgbm import LGBMClassifier
# lgb = LGBMClassifier()
# scores = cross_validate(lgb, X_train, y_train, cv=splitter)
# print(np.mean(scores['test_score']))

In [ ]:
# from xgboost import XGBClassifier
# xgb = XGBClassifier()
# scores = cross_validate(xgb, X_train, y_train, cv=splitter)
# print(np.mean(scores['test_score']))

In [20]:
df3 = pd.read_csv(path+file)
df3.dropna(subset=['결과'], inplace=True)
df3['morph_str'] = df3['사람문장1'].apply(morph_list)

data_text3 = df3['morph_str']
data_senti3 = df3['결과']

train_data_text3, test_data_text3, train_data_senti3, test_data_senti3 = \
train_test_split(data_text3, data_senti3, stratify=data_senti3, \
                 train_size=0.75, random_state=1111)
test_data_senti_freq3 = Counter(test_data_senti3)

# print('test_data_senti_freq:', test_data_senti_freq3)
vect3 = CountVectorizer(min_df=5).fit(train_data_text3)
X_train3 = vect.transform(train_data_text3)
X_test3 = vect.transform(test_data_text3)
# print("X_train3:\n", repr(X_train3))

In [22]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression

y_train3 = pd.Series(train_data_senti3)
y_test3 = pd.Series(test_data_senti3)

splitter = StratifiedKFold(n_splits=4, shuffle=True, random_state=1111)
scores3 = cross_validate(lr, X_train3, y_train3, cv=splitter)
print('score3:', np.mean(scores3['test_score']))

lr.fit(X_train3, y_train3)
print('test_score: ', lr.score(X_test3, y_test3))

score3: 0.7927305198373448
test_score:  0.7922222222222223


In [51]:
result3 = lr.predict(X_test3)
result3 = pd.Series(result)
result3.value_counts()

-1.0    819
 1.0     64
 0.0     17
dtype: int64

In [52]:
result = lr.predict(X_test)
result = pd.Series(result)
result.value_counts()

-1.0    726
 1.0     61
 0.0     11
dtype: int64

In [15]:
len(svc_result)

798

In [16]:
test_data_text.head()

1546       젊다 운동 많이 두다 아직 걷다 생활 무리
3404             아들 학교 폭력 연루 확실 교육
2452    신장 투석 매번 번거롭다 아프다 너무 짜증 나다
2538      아내 어제 아무 그냥 볼 계속 노려보다 불안
3430          믿기다 평소 정말 친하다 친구 친하다
Name: morph_str, dtype: object

In [17]:
test_data_senti.head()

1546    1.0
3404   -1.0
2452   -1.0
2538   -1.0
3430    1.0
Name: 결과, dtype: float64

In [18]:
len(test_data_text)

798

In [19]:
data_text = test_data_text
# 긍정단어사전(최종).csv
# 긍정단어사전(최종).csv
positive = pd.read_csv(path+'긍정단어사전(최종).csv', encoding='cp949')
positive.iloc[-1] = '환갑'
positive = positive['환갑'].to_list()

negative = pd.read_csv(path+'부정단어사전(최종).csv', encoding='cp949')
negative.iloc[-1] = '신경'
negative = negative['신경'].to_list()

In [20]:
def cntWorldInLine(data, senti):
    senti_found = []
    for onedata in data:
        senti_temp = 0                        # 그 줄에서 발견된 감정단어의 수를 담는 변수
        for sentiword in senti:               # 감정 사전의 어휘
            if sentiword[0] in onedata:  #sentiword[0] 하여 리스트 원소를 추출(문자열)
                senti_temp += 1               # 현재의 감정단어와 일치하면 숫자를 하나 올려 줌.(중복허용 X)
        senti_found.append(senti_temp)        # 현재의 줄에서 찾은 감성단어의 숫자를 해당 위치에 저장
    return senti_found

data_senti_poscnt = cntWorldInLine(data_text, positive)
data_senti_negcnt = cntWorldInLine(data_text, negative)

In [27]:
new_df = pd.DataFrame({'tag':y_test, 'morph':data_text, 
                       'pos':data_senti_poscnt, 'neg':data_senti_negcnt })
new_df.head()

,tag,morph,pos,neg
1546,1.0,젊다 운동 많이 두다 아직 걷다 생활 무리,12,8
3404,-1.0,아들 학교 폭력 연루 확실 교육,1,8
2452,-1.0,신장 투석 매번 번거롭다 아프다 너무 짜증 나다,9,15
2538,-1.0,아내 어제 아무 그냥 볼 계속 노려보다 불안,9,16
3430,1.0,믿기다 평소 정말 친하다 친구 친하다,7,5


In [28]:
new_df['senti_score'] = new_df['pos'] - new_df['neg']
new_df.loc[new_df['senti_score'] > 0, 'new'] = 1
new_df.loc[new_df['senti_score'] <= 0, 'new'] = -1

In [29]:
new_df.loc[pd.to_numeric(new_df.tag) == new_df.new, 'matched']= True
new_df.loc[pd.to_numeric(new_df.tag) != new_df.new, 'matched'] = False

In [30]:
new_df.head(2)

,tag,morph,pos,neg,senti_score,new,matched
1546,1.0,젊다 운동 많이 두다 아직 걷다 생활 무리,12,8,4,1.0,True
3404,-1.0,아들 학교 폭력 연루 확실 교육,1,8,-7,-1.0,True


In [31]:
matched = sum(new_df['matched'])
mismatched = len(new_df) - matched

score = matched / len(new_df) * 100
score

64.03508771929825